# Figures associated to immune processing

**You need to tun the notebook 2_ first!!!!**

In [ ]:
from cellassign import assign_cats
import gzip
import itertools as itl
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import seaborn as sns
import triku as tk
import itertools
import matplotlib.pylab as pylab

In [ ]:
from functions import adata_plot_KOvsWT, stat_annot_gene
from functions import dict_WT_KO_colors

In [ ]:
from datetime import date

today = date.today()
today = today.strftime("%Y_%m_%d")

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
params = {'figure.dpi': 200,
          'legend.fontsize': 'large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large', 
         'axes.grid': False}
pylab.rcParams.update(params)
# mpl.style.use('default')

pd.set_option('display.max_columns', None)

In [ ]:
os.makedirs('results/figures/F3', exist_ok=True)

## Functions and variables

In [ ]:
%store -r dict_WT_KO_colors
%store -r seed

## Load adata

In [ ]:
adata_all_harmony = sc.read_h5ad('adatas/adata_all_harmony.h5')
adata_imm = adata_all_harmony[adata_all_harmony.obs['merged_cell_type_name'].isin(['APC', 'LYM', 'NEU', 'MC'])]

In [ ]:
sc.pp.filter_genes(adata_imm, min_counts=1)

In [ ]:
sc.pp.pca(adata_imm, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_imm, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_imm, random_state=seed, n_neighbors=int(len(adata_imm) ** 0.5 * 0.5), metric='cosine', use_rep='X_pca_harmony')
tk.tl.triku(adata_imm, use_raw=False)

sc.pp.pca(adata_imm, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_imm, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_imm, random_state=seed, n_neighbors=int(len(adata_imm) ** 0.5 * 0.5), metric='cosine', use_rep='X_pca_harmony')

In [ ]:
sc.tl.umap(adata_imm, min_dist=1)

### Subpopulation annotation

To do the annotation we used the characterization made by Joost et al. 2020. There are some populations that are not appearing (Krt75+ to Krt18+) and we are not sure about their functional relationship

In [ ]:
dict_immune = {  '0A: LC': ['Mfge8', 'Cd207', 'Cldn1', 'Pgf', 'Cd74', 'H2-Aa', 'H2-Eb1',  'Nr4a3', 'Il1r2', 'Pkib', 'Grk3', 'Ktn1', 'Pxdc1', 'Dnah2', 'Timd4', 
                                      'Epcam', 'Ccr7', 'St14', 'Mreg', 'Cd40'],
                 '0B: CCR7$^+$ LC': ['Fscn1', 'Cd200', 'Il4i1', 'Pkib', 'Wnk1', 'Cd74', 'Cst3', 'Ccr7', 'Bcl2a1a', 'Cacnb3', 'Ccl22', 'Gm13546', 'Serpinb9b', 'Relb', 'Cd40', 'Arl5a'],

                 '1A: Mφ/DCA': ['Pf4', 'C1qb', 'Igf1', 'Grn', 'Dab2', 'C1qc', 'Apoe', 'C1qa', 'Ms4a7', 'Timp2', 'Cd36', 'Mrc1', 'Fxyd2', 'F13a1', 'Rgl1', 'Ednrb', 'Hgsnat', 'Tgfbi', 'Rnase4', 
                                 'Sash1', 'Folr2', 'Gas6', 'Stab1', 'Ccl8', 'Fcgrt', 'Ms4a6c', 'P2rx7'] ,
                '1B: Mφ/DCB': ['Cd209a', 'Cd209d', 'Slc39a14', 'Plbd1', 'Aif1', 'Cd74', 'Clec4e', 'Tgfbi', 'Pdpn', 'Cxcl3', 'Vat1', 'Eps8', 'Tppp3', 
                                     'Procr', 'Tarm1', 'Clec4a3', 'Ccl5'] ,
               '1C: Mφ/DCC': ['Nos2', 'Saa3', 'Arg1', 'Wfdc17', 'Csf1r', 'F10', 'Ly6c2', 'Gsr', 'Anxa5', 'Ptgs2', 'Cxcl3', 'Pid1', 'Serpinb2', 'Csf3', 'Vcan', 'Inhba', 'Ptges', 
                               'F13a1', 'Met', ],
    
                 '2A: Act. Neu': ['Acod1', 'Il1f9', 'Csta3', 'Rhov', 'Stfa2l1', 'Ccl3', 'Hcar2', 'Slpi', 'Ccrl2', 'Il1rn', 'Cxcl3', 'G0s2', 
                                      'Stfa2', 'Ptgs2', 'Asprv1', 'Fpr1'],
    
                 '2B: Neu': ['Camp', 'Ltf', 'Chil3', 'S100a8', 'S100a9', 'Tinagl1', 'Stfa1', 'Ngp', 'Chil3', 'Cd177', 'Adpgk', 'Syne1', 'Cebpe', 'Hsd11b1', 'Mgst1', 
                                 'Tst', 'Ly6c2', 'Pilrb2', 'Fcnb', 'Rab44', 'Nfe2', 'Cpne3', 'Ldhc', 'Hmgn2', 'Adgrg3'],
               
                 '3: Bas': ['Mcpt8', 'Cyp11a1', 'Cd200r3', 'Ifitm1', 'Ero1l', 'Ccl6', 'Ccl3', 'Lilrb4a', 'Gata2', 'Ccl4', 'Hdc', 'Osm', 'Lat2', 'Tec', 'Rab44', 'Csf2rb2', 
                               'Os9', 'Ier3', 'Aqp9', 'Cd200r4', 'Emilin2', 'Csf1', 'Slpi', 'Hmox1', 'Mboat1', 'Hs3st1', 'Adgrg3', ],
                 
                 '4: BC': ['Vpreb3', 'Cd79a', 'Cd79b', 'Chchd10', 'Ighm',  'Stmn1', 'Igll1', 'Dut', 'Top2a', 'Rasgrp2', 'Myb', 'Ssrp1', 'Gimap6', 'Selenoh', 'Apobec3', 'Smc2',
                             'Cks1b', 'Lgals9', 'Ebf1', 'Csrp2', 'Adgrg1', 'Pou2af1', 'Tcf3', 'Hist1h1a', 'Parp1', 'Lig1', 'Dntt', 'Sox4'],
               
                 '5: PC': ['Iglc3', 'Ms4a1', 'Iglc2', 'Ighm', 'Igkc', 'Cd79a', 'Ly6d', 'Cd79b', 'Tnfrsf13c', 'Iglc1', 'Sdc1', 'Cd74', 
                                  'Txndc16', 'Ighd'],    
                  '6: αβT': ['Tnfrsf18', 'Cxcr6', 'Ccr2', 'Ifngr1', 'Cd52', 'Rora', 'Il7r', 'Aebp2', 'Icos', 'Dgat1', 'Shisa5', 'Grap', 'Cd27', 'Pdcd1', 'Ikzf3', 'Il18r1', 
                             'Gata3', 'Spn', 'Tnfrsf4'],
                  '7: γδT': ['Nkg7', 'Ctsw', 'Cd7', 'Rab4a', 'Ctla2a', 'Rgs2', 'Cd3d', 'Cd3g', 'Fermt2', 'Tcrg-C1', 'Klrk1', 'Trf', 'Itk', 'Ubald2', 'Il2rb', 'Map1b', 'Klrb1b', 'Cd2', 
                              'Trbc2', 'Abcb1b', 'Rasl11b', 'Parp8', 'Tiam1'],
                  '8: MC': ['Cma1', 'Tpsb2', 'Mcpt4', 'Fdx1', 'Hs3st1', 'Cd55', 'Tpsab1', 'Hdc', 'Tph1', 'Cpa3', 'Fgfr1', 'Gata2', 'Slco2b1', 'Vwa5a', 'Il1rl1', 
                                'Cyp11a1', 'Adora3', 'Kit', 'Ap1s3', 'Hexb', 'Creb3l1', 'Mrgprb1', 'Rab27b', 'Penk', 'Hs6st2', 'Adamts9', 'Akap12', 'Slc45a3', 'Mrgprb2', 'Gnai1'], 
                  }



dict_imm_colors = { '0: LC': '#D40055',
#                     '1: CCR7$^+$ LC': '#f25a97',
                    '1: Mφ/DC': '#4D0765',
#                     '2: Act. Neu': '#D3BC5F',
                    '2: Neu': '#A5901A',
                    '3: Bas': '#A05A2C',
                    '4: BC': '#4c708a',
                    '5: PC': '#00171F',
                    '6: αβT': '#0086b9',
                    '7: γδT': '#99DCF5',
                    '8: MC': '#FF9955',
                     }


In [ ]:
sc.tl.leiden(adata_imm, resolution=4)
assign_cats(adata_imm, dict_cats=dict_immune, min_score=0.4, quantile_gene_sel=0.6, key_added='subtype')

adata_imm.obs['subtype'] = [i  if i[0] != '0' else '0: LC' for i in adata_imm.obs['subtype']]
adata_imm.obs['subtype'] = [i  if i[0] != '1' else '1: Mφ/DC' for i in adata_imm.obs['subtype']]
adata_imm.obs['subtype'] = [i  if i[0] != '2' else '2: Neu' for i in adata_imm.obs['subtype']]


adata_imm.obs['subtype_name'] = [i.split(': ')[1] for i in adata_imm.obs['subtype']]
adata_imm.obs['subtype_name'] = adata_imm.obs['subtype_name'].astype('category')
adata_imm.obs['subtype_name'].cat.reorder_categories([i.split(': ')[1] for i in dict_imm_colors.keys()], inplace=True)

adata_imm.obs['subtype_number'] = [i.split(': ')[0] for i in adata_imm.obs['subtype']]
adata_imm.obs['subtype_number'] = adata_imm.obs['subtype_number'].astype('category')
adata_imm.obs['subtype_number'].cat.reorder_categories([i.split(': ')[0] for i in dict_imm_colors.keys()], inplace=True)


adata_imm.uns['subtype_colors'] = list(dict_imm_colors.values())
adata_imm.uns['subtype_name_colors'] = list(dict_imm_colors.values())
adata_imm.uns['subtype_number_colors'] = list(dict_imm_colors.values())

## UMAP of fibroblast subpopulations and batch

#### Supp Fig to Fig 3

In [ ]:
adata_imm_WT = adata_imm[adata_imm.obs['batch'].isin(['WT1', 'WT2'])]
adata_imm_WT.uns['batch_colors'] = [dict_WT_KO_colors['WT1'], dict_WT_KO_colors['WT2'], ]
adata_imm_KO = adata_imm[adata_imm.obs['batch'].isin(['KO1', 'KO2'])]
adata_imm_KO.uns['batch_colors'] = [dict_WT_KO_colors['KO1'], dict_WT_KO_colors['KO2'], ]

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(8, 4))
s = 18

sc.pl.umap(adata_imm, color='subtype_number', s=s, ax = axs, 
           legend_loc='on data', frameon=False, show=False, 
           legend_fontoutline=4, legend_fontsize=14)
sc.pl.umap(adata_imm, color='subtype', s=s, ax = axs, 
           frameon=False, show=False)

title_obj = axs.set_title('') 

plt.tight_layout()

plt.savefig(f'results/figures/F3/makers_imm_closeup_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)
plt.savefig(f'results/figures/F3/Supp_Fig3A_closeup_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 7))
s = 18


sc.pl.umap(adata_imm_WT, color='batch', s=s, ax = axs.ravel()[0], 
           frameon=False, show=False, na_in_legend=False)


sc.pl.umap(adata_imm_WT, color='subtype_number', s=s, ax=axs.ravel()[1], 
           legend_loc='on data', frameon=False, show=False, legend_fontoutline=4, legend_fontsize=14, 
           na_in_legend=False, title='condition')
sc.pl.umap(adata_imm_WT, color='subtype', s=s, ax = axs.ravel()[1], 
           frameon=False, show=False, na_in_legend=False, title='FB type')



sc.pl.umap(adata_imm_KO, color='batch', s=s, ax = axs.ravel()[2], 
           frameon=False, show=False, na_in_legend=False, title='')


sc.pl.umap(adata_imm_KO, color='subtype_number', s=s, ax=axs.ravel()[3], 
           legend_loc='on data', frameon=False, show=False, legend_fontoutline=4, legend_fontsize=14, 
           na_in_legend=False, title='')


plt.tight_layout()

plt.savefig(f'results/figures/F3/makers_imm_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)
plt.savefig(f'results/figures/F3/Supp_Fig3A_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)

## log2fold heatmap highlighting the expression of specific markers

In [ ]:
markers = {'Suppresive': ['Cd274', 'Pdcd1lg2', 'Cd80', 'Lgals9', 'Lgals3', 'Cd74', 
                          'Adora2a', 'Adora2b', 'Arg1', 'Arg2', 'Il4i1',],  
           'Stimulatory': ['Il10', 'Tnfsf9', 'Tnfsf4', 'Cd86',], 
           'Proinflammatory cytokines': ['Il1a', 'Il1b', 'Il1rn', 'Tnf', 'Il6', 'Ifng', 
              'Tgfb1',],
           'IL1 processing': ['Nlrp3', 'Casp1', 'Casp8', 'Gzmb', ], 
           'Chemoattractant': ['Ccl3', 'Ccl4', 'Ccl5', 'Ccl17', 'Ccl22', 'Cxcl1', 
                                'Cxcl2', 'Cxcl3', 'Cxcl16', 'Csf3',],
           'Chemoreceptors': ['Ccr1', 'Ccr2', 'Ccr4', 'Ccr7', 'Cxcr6', 'C3ar1']
           
          }

In [ ]:
list_imm_logfold_genes = [j for i in markers.values() for j in i]

categories_order = adata_imm.obs['subtype_name'].cat.categories
categories_order = [i for i in categories_order if i != 'Bas']
# There are no basophil cells in WT, so the logfold is 0

df_logfold = pd.DataFrame(columns=categories_order, index=list_imm_logfold_genes)
df_mean = pd.DataFrame(columns=categories_order, index=list_imm_logfold_genes)

for cell_type in categories_order:
    adata_WT = adata_imm_WT[adata_imm_WT.obs['subtype_name'] == f"{cell_type}"]
    adata_KO = adata_imm_KO[adata_imm_KO.obs['subtype_name'] == f"{cell_type}"]
    
    mean_WT = adata_WT[:, ].X.mean(0).A1
    mean_KO = adata_KO[:, ].X.mean(0).A1
    
    # Creating a df and selecting the genes is actually faster than selecting them with scanpy
    df = pd.DataFrame({'WT': mean_WT, 'KO':mean_KO}, index=adata_imm.var_names)
    df = df.loc[list_imm_logfold_genes]

    min_val = 0.1

    logfold = np.log2((df['KO'] + min_val) / (df['WT'] + min_val))
    mean = (df['KO'] + df['WT']) / 2
    
    df_logfold.loc[list_imm_logfold_genes, cell_type] = logfold
    df_mean.loc[list_imm_logfold_genes, cell_type] = mean

    df_logfold = df_logfold.astype(float)
    df_mean = df_mean.astype(float)

In [ ]:
list_labels = ['Suppresive', 'Stim.', 'Proinf.', 'IL1 proc.', 'Chemoatt.', 'Chemorec.']

pylab.rcParams.update(params)

fig, ax = plt.subplots(1, 1, figsize=(4*1.5, 7*1.5))
text_ax = fig.add_axes([.9, 0.125, .02, 0.755], sharey=ax)
cbar_ax = fig.add_axes([.98, .58, .03, .2])


sns.heatmap(df_logfold, ax=ax, cbar_ax = cbar_ax, cbar=True, cmap='RdBu_r', center=0, 
            yticklabels=True, vmax=4, vmin=-2)

# Plot the categories
counter = 0
x_bar = len(categories_order)
text_ax.tick_params(labelbottom=False, labelleft=False)
text_ax.set_axis_off()

for idx, (category, genes) in enumerate(markers.items()):
    text_ax.plot([0, 0], [counter + 0.25, counter + len(genes) - 0.25], c="#404040")
    text_ax.text(x=0.1, y = counter + len(genes)/2, s=list_labels[idx], rotation=-90, 
                 ha='center', va='center', size='large')
    counter += len(genes)
    
    
plt.savefig(f'results/figures/F3/matrixplot_selected_markers_{date.today().strftime("%Y_%m_%d")}_HEATMAP_LOG2.jpg', dpi=300, bbox_inches="tight")
plt.savefig(f'results/figures/F3/Fig3C.jpg', dpi=300, bbox_inches="tight")

In [ ]:
adata_T_cells = adata_imm[adata_imm.obs['subtype_name'].isin(['αβT', 'γδT'])]
adata_T_cells.obs['subtype-condition'] = adata_T_cells.obs['subtype_name'].astype(str) + '-' + adata_T_cells.obs['condition'].astype(str)

In [ ]:
t_genes = ['Cd3g', 'Trac', 'Tcrg-C1', 'Trdv4', 'Nkg7', 'Cd4', 'Cd8a', 'Foxp3', 'Il2rb',
           'Il2rg', 'Il7r', 'Pdcd1', 'Ctla4', 'Tigit', 'Havcr2', 'Lag3', 'Tnfrsf18', 
           'Tnfrsf4', 'Tnfrsf9', 'Lef1', 'Sell', ]

other_t_genes = ['Trdc',  'Cd163l1', 'Il2ra',  'Tcf7',  'Prf1', 'Gzma', 'Gzmb']
sc.pl.heatmap(adata_T_cells, var_names=t_genes, groupby='subtype-condition', use_raw=False, swap_axes=True, show=False)

plt.savefig(f'results/figures/F3/heatmap_T_markers_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300, bbox_inches="tight")
plt.savefig(f'results/figures/F3/Fig3B.jpg', dpi=300, bbox_inches="tight")

In [ ]:
sc.pl.matrixplot(adata_T_cells, var_names=t_genes, groupby='subtype-condition', use_raw=False, show=False)

plt.savefig(f'results/figures/F3/matrixplot_T_markers_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300, bbox_inches="tight")

### Analysis os KO vs WT in ab and gd T cells

In [ ]:
sc.tl.rank_genes_groups(adata_T_cells, groupby='subtype-condition', groups=['γδT-KO'], 
                       reference='γδT-WT')

In [ ]:
for gene in adata_T_cells.uns['rank_genes_groups']['names']['γδT-KO'][:80]:
    sc.pl.umap(adata_T_cells[adata_T_cells.obs['condition'] == 'WT'], 
               color=['subtype-condition', gene], cmap=magma, use_raw=False, ncols=4)
    sc.pl.umap(adata_T_cells[adata_T_cells.obs['condition'] == 'KO'], 
               color=['subtype-condition', gene], cmap=magma, use_raw=False, ncols=4)

In [ ]:
genesgd = ['Aldoa', 'Tpi1', 'Eno1', 'Ldha', 'Mif', 'Pgk1', 'Gpi1', 'Itm2c', 'Pgam1', 'Col7a1', 
           'Ypel3', 'Pde7a', 'Usp48', 'Bin1', 'Ssbp2', 'Cflar', 'Slfn2']

sc.pl.matrixplot(adata_T_cells, var_names=genesgd, groupby='subtype-condition', use_raw=False, show=False)
plt.savefig(f'results/figures/F3/matrixplot_DEG_gdT_markers_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300, bbox_inches="tight")
sc.pl.heatmap(adata_T_cells, var_names=genesgd, groupby='subtype-condition', use_raw=False, swap_axes=True, show=False)
plt.savefig(f'results/figures/F3/heatmap_DEG_gdT_markers_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300, bbox_inches="tight")


In [ ]:
sc.tl.rank_genes_groups(adata_T_cells, groupby='subtype-condition', groups=['αβT-KO'], 
                       reference='αβT-WT')

In [ ]:
for gene in adata_T_cells.uns['rank_genes_groups']['names']['αβT-KO'][:80]:
    sc.pl.umap(adata_T_cells[adata_T_cells.obs['condition'] == 'WT'], 
               color=['subtype-condition', gene], cmap=magma, use_raw=False, ncols=4)
    sc.pl.umap(adata_T_cells[adata_T_cells.obs['condition'] == 'KO'], 
               color=['subtype-condition', gene], cmap=magma, use_raw=False, ncols=4)

In [ ]:
genesab = ['Aldoa', 'Gpi1', 'Eno1', 'Pgk1', 'Ly6e', 'Tnfrsf9', 'Tpi1', 'Il7r', 
            'Sdf4', 'Shisa5', 'Gmfg', 'Sept1', 'Hif1a', 'Col7a1', 'Tet2', 
            'Stk26', 'Scamp3', 'Trabd', 'Apobec3', 'Vma21', ]

sc.pl.matrixplot(adata_T_cells, var_names=genesab, groupby='subtype-condition', use_raw=False, show=False)
plt.savefig(f'results/figures/F3/matrixplot_DEG_abT_markers_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300, bbox_inches="tight")
sc.pl.heatmap(adata_T_cells, var_names=genesab, groupby='subtype-condition', use_raw=False, swap_axes=True, show=False)
plt.savefig(f'results/figures/F3/heatmap_DEG_abT_markers_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300, bbox_inches="tight")


##### Analysis of ab T cells

In [ ]:
adata_T_cells.obs['subtype_name'] == 'αβT'

In [ ]:
[i for i in adata_T_cells.var_names if 'Havcr' in i]

In [ ]:
ab_t_genes = ['Cd3g', 'Trac', 'Trbc2', 'Cd4', 'Cd8a', 'Foxp3', 'Il2ra', 'Il7r', 
              'Pdcd1', 'Tcf7', 'Slamf6', 'Tox', 'Nfatc2', 'Nr4a1', 'Ctla4', 
              'Havcr2', 'Havcr1', 'Tigit', 'Lag3', 'Gzmb', 'Tnfrsf18', 'Tnfrsf9', 'Tnfrsf4', ]

#Havcr4 is not -> add Havcr1 and Havcr2
sc.pl.heatmap(adata_T_cells[adata_T_cells.obs['subtype_name'] == 'αβT'], 
              var_names=ab_t_genes, groupby='subtype-condition', use_raw=False, swap_axes=True, show=False)

plt.savefig(f'results/figures/F3/heatmap_abT_markers_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300, bbox_inches="tight")

In [ ]:
sc.pl.matrixplot(adata_T_cells[adata_T_cells.obs['subtype_name'] == 'αβT'], 
                 var_names=ab_t_genes, groupby='subtype-condition', use_raw=False, show=False)

plt.savefig(f'results/figures/F3/matrixplot_abT_markers_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300, bbox_inches="tight")

### Analysis of KO vs WT in Neutrophils cells

In [ ]:
adata_neu = adata_imm[adata_imm.obs['subtype_name'] == 'Neu']
adata_neu.obs['subtype-condition'] = adata_neu.obs['subtype_name'].astype(str) + '-' + \
                                     adata_neu.obs['condition'].astype(str)

In [ ]:
sc.tl.rank_genes_groups(adata_neu, groupby='subtype-condition', groups=['Neu-KO'], 
                       reference='Neu-WT')

In [ ]:
for gene in adata_neu.uns['rank_genes_groups']['names']['Neu-KO'][:80]:
    sc.pl.umap(adata_neu[adata_neu.obs['condition'] == 'WT'], 
               color=['subtype-condition', gene], cmap=magma, use_raw=False, ncols=4)
    sc.pl.umap(adata_neu[adata_neu.obs['condition'] == 'KO'], 
               color=['subtype-condition', gene], cmap=magma, use_raw=False, ncols=4)

In [ ]:
genes_neu = ['Hilpda', 'Cxcl2', 'Ccl3', 'Ier3', 'Nfkbia', 
             'Csta3', 'Hcar2', 'Slpi', 'Mif', 'Il1b', 'Smox', 'Cstb', 'Pim1', 
             'Ccrl2', 'Il1rn', 'Cxcl3', 'Ankrd33b', 'Ptgs2', 'Asprv1', 
#              'Stfa2l1', 'G0s2', 'Stfa2',
             'Col7a1', 'Basp1', 'Pfkp', 'Ptgs2', 
             'Upp1', 'Card19', 'Nlrp3', 
             'Cd274', 'Emilin2',  'Tnfaip3', 'Il1f9',
             ] \

sc.pl.matrixplot(adata_neu, var_names=genes_neu, groupby='subtype-condition', 
                 use_raw=False, show=False, vmax=3)
plt.savefig(f'results/figures/F3/matrixplot_DEG_neu_markers_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300, bbox_inches="tight")
sc.pl.heatmap(adata_neu, var_names=genes_neu, groupby='subtype-condition', 
              use_raw=False, swap_axes=True, show=False, vmax=3)
plt.savefig(f'results/figures/F3/heatmap_DEG_neu_markers_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300, bbox_inches="tight")

### Analysis os KO vs WT in Langerhans cells

In [ ]:
adata_LC = adata_imm[adata_imm.obs['subtype_name'] == 'LC']
adata_LC.obs['subtype-condition'] = adata_LC.obs['subtype_name'].astype(str) + '-' + \
                                     adata_LC.obs['condition'].astype(str)

In [ ]:
sc.tl.rank_genes_groups(adata_LC, groupby='subtype-condition', groups=['LC-KO'], 
                       reference='LC-WT')

In [ ]:
for gene in adata_LC.uns['rank_genes_groups']['names']['LC-KO'][:80]:
    sc.pl.umap(adata_LC[adata_LC.obs['condition'] == 'WT'], 
               color=['subtype-condition', gene], cmap=magma, use_raw=False, ncols=4)
    sc.pl.umap(adata_LC[adata_LC.obs['condition'] == 'KO'], 
               color=['subtype-condition', gene], cmap=magma, use_raw=False, ncols=4)

In [ ]:
genes_LC = ['Actg1', 'Aldoa', 'Anxa2',  'Pgk1', 'Tnfrsf9', 'Mif', 
            'Ldha', 'Tpi1', 'Ccr7', 'Abracl','Pkib', 'Sin3b', 
            'Rhoc', 'Samsn1', 'Il4i1', 'Bcl2a1a',
             'Ktn1', 'Eno1', 'Slfn2', 'Znrf1', 
            'Sri', 'Syngr2',  'Rabgap1l', 'Tank', 
            'Lims1', 'Timd4', 'Cacnb3', 'Sugt1', 'Jak2',  
            'Stk24', 'Plxnd1', 'Batf3', 
            'Lrp1', 'Cyfip1',  'Rnf19b', 'Ddit4',    
             ]

sc.pl.matrixplot(adata_LC, var_names=genes_LC, groupby='subtype-condition', use_raw=False, 
                 show=False, vmax=2.5)
plt.savefig(f'results/figures/F3/matrixplot_DEG_LC_markers_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300, bbox_inches="tight")
sc.pl.heatmap(adata_LC, var_names=genes_LC, groupby='subtype-condition', use_raw=False, 
              swap_axes=True, show=False, vmax=2.5)
plt.savefig(f'results/figures/F3/heatmap_DEG_LC_markers_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300, bbox_inches="tight")
